In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


In [ ]:
df=pd.read_csv('/content/Question_Classification_Dataset.csv')
df.head()

,Unnamed: 0,Questions,Category0,Category1,Category2
0,0,How did serfdom develop in and then leave Russ...,DESCRIPTION,DESC,manner
1,1,What films featured the character Popeye Doyle ?,ENTITY,ENTY,cremat
2,2,How can I find a list of celebrities ' real na...,DESCRIPTION,DESC,manner
3,3,What fowl grabs the spotlight after the Chines...,ENTITY,ENTY,animal
4,4,What is the full form of .com ?,ABBREVIATION,ABBR,exp


In [ ]:
df["Category0"].unique()

array(['DESCRIPTION', 'ENTITY', 'ABBREVIATION', 'HUMAN', 'NUMERIC',
       'LOCATION'], dtype=object)

In [ ]:
df.columns

Index(['Unnamed: 0', 'Questions', 'Category0', 'Category1', 'Category2'], dtype='object')

In [ ]:
le = LabelEncoder()
df['encoded'] = le.fit_transform(df['Category0'])


In [ ]:
df.columns = df.columns.str.strip()


In [ ]:
df_final=df[['Questions','Category0']]
df_final.head()

,Questions,Category0
0,How did serfdom develop in and then leave Russ...,DESCRIPTION
1,What films featured the character Popeye Doyle ?,ENTITY
2,How can I find a list of celebrities ' real na...,DESCRIPTION
3,What fowl grabs the spotlight after the Chines...,ENTITY
4,What is the full form of .com ?,ABBREVIATION


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_final["Questions"], df["Category0"], test_size=0.2)


In [ ]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

ABBREVIATION       0.00      0.00      0.00        19
 DESCRIPTION       0.72      0.69      0.71       228
      ENTITY       0.64      0.73      0.68       258
       HUMAN       0.67      0.87      0.75       230
    LOCATION       0.94      0.66      0.78       181
     NUMERIC       0.88      0.78      0.82       175

    accuracy                           0.73      1091
   macro avg       0.64      0.62      0.62      1091
weighted avg       0.74      0.73      0.73      1091



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Hugging  face transformation**

In [ ]:
pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_final_2[["Questions", "encoded"]])


dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
# Ensure the 'encoded' column is included and renamed to 'labels' in the tokenized dataset
def tokenize(example):
    tokenized_inputs = tokenizer(example["Questions"], truncation=True, padding="max_length")
    tokenized_inputs["labels"] = example["encoded"] # Include and rename the labels
    return tokenized_inputs

tokenized = dataset.map(tokenize, batched=True)

# The rest of the code remains the same
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(le.classes_)
)

from google.colab import drive
drive.mount('/content/drive')

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/models/results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
def predict_question(question):
    inputs = tokenizer(question, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax().item()
    return le.inverse_transform([predicted_class])[0]

# Example
print(predict_question("What are the top-selling items last month?"))  # → "Top-selling"
